In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Enumerate all possible N=2 random walks

In [3]:
directions = [(1,0), (0,1), (-1,0), (0,-1)]

In [4]:
rows = []
for dx1, dy1 in directions:
    for dx2, dy2 in directions:
        x = dx1 + dx2
        y = dy1 + dy2
        rows.append({'x': x, 'y': y})
df = pd.DataFrame(rows)

In [5]:
df

,x,y
0,2,0
1,1,1
2,0,0
3,1,-1
4,1,1
5,0,2
6,-1,1
7,0,0
8,0,0
9,-1,1


## Enumerate all possible N=4 random walks

In [6]:
rows = []
for dx1, dy1 in directions:
    for dx2, dy2 in directions:
        for dx3, dy3 in directions:
            for dx4, dy4 in directions:
                x = dx1 + dx2 + dx3 + dx4
                y = dy1 + dy2 + dy3 + dy4
                rows.append({'x': x, 'y': y})
df = pd.DataFrame(rows)

In [7]:
df

,x,y
0,4,0
1,3,1
2,2,0
3,3,-1
4,3,1
...,...,...
251,-1,-3
252,1,-3
253,0,-2
254,-1,-3


This approach will not scale to larger values of $N$

Instead, consider each particular walk a path through a tree data structure.

<img width="25%" src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/5f/Tree_%28computer_science%29.svg/1200px-Tree_%28computer_science%29.svg.png" alt="Tree (computer science).svg">

<a href="https://commons.wikimedia.org/wiki/File:Tree_(computer_science).svg" title="via Wikimedia Commons">Paddy3118</a> 
[<a href="https://creativecommons.org/licenses/by-sa/4.0">CC BY-SA</a>]

Each node will have 4 children indicating a direction to take. The number of levels in the tree depends on the number of steps to take.

Traversing tree-like data structures are examined in computer science.  Here we review a technique called a *Depth First Transveral*. 

Here, each node represents the position (x, y) at a particular step along the random walk. Say we want to find all possible walks up to $N=4$.

In [8]:
N = 4

In [9]:
rootNode = {'n': 0, 'x': 0, 'y': 0}

In [10]:
directions = [(1,0), (0,1), (-1,0), (0,-1)]

In [11]:
def walk_tree(node):
    if node['n'] == N:
        yield node
        return

    for dx, dy in directions:
        newNode = {'x': node['x']+dx, 
                   'y': node['y']+dy, 
                   'n': node['n']+1}

        yield from walk_tree(newNode)

In [12]:
pd.DataFrame(walk_tree(rootNode))

,x,y,n
0,4,0,4
1,3,1,4
2,2,0,4
3,3,-1,4
4,3,1,4
...,...,...,...
251,-1,-3,4
252,1,-3,4
253,0,-2,4
254,-1,-3,4


Is recursion necessary? No, we could use a stack instead.

In [13]:
def walk_tree(rootNode):
    
    stack = [rootNode]
    while len(stack) > 0:
        node = stack.pop()
        
        if node['n'] == N:
            yield node
            continue
        
        for dx, dy in directions:
            newNode = {'x': node['x']+dx, 
                       'y': node['y']+dy, 
                       'n': node['n']+1 }
            stack.append(newNode)

In [14]:
pd.DataFrame(walk_tree(rootNode))

,x,y,n
0,0,-4,4
1,-1,-3,4
2,0,-2,4
3,1,-3,4
4,-1,-3,4
...,...,...,...
251,3,1,4
252,3,-1,4
253,2,0,4
254,3,1,4


Another approach is to keep track of which direction we need to head next for each level. Here we are not tracking individual walks but only counting how many walks there are of each size.

In [20]:
def walk_tree(N=4):
    
    dir = np.zeros(N+1)
    count = np.zeros(N+1)
    
    n = 0
    
    while True:
        if dir[n] == 4:
            n = n - 1
            if n == 0:
                break
        else:
            count[n] += 1
            dir[n] += 1
            
            if n < N:
                n = n + 1
                dir[n] = 0
    return count

In [35]:
%%time
walk_tree(N=3)

CPU times: user 11 µs, sys: 0 ns, total: 11 µs
Wall time: 12.4 µs


array([ 1.,  4., 16., 64.])

In [36]:
%%time
walk_tree(N=10)

CPU times: user 2.52 ms, sys: 0 ns, total: 2.52 ms
Wall time: 2.52 ms


array([1.000000e+00, 4.000000e+00, 1.600000e+01, 6.400000e+01,
       2.560000e+02, 1.024000e+03, 4.096000e+03, 1.638400e+04,
       6.553600e+04, 2.621440e+05, 1.048576e+06])

In [40]:

%%time
walk_tree(N=15)

CPU times: user 2.72 s, sys: 0 ns, total: 2.72 s
Wall time: 2.71 s


array([1.00000000e+00, 4.00000000e+00, 1.60000000e+01, 6.40000000e+01,
       2.56000000e+02, 1.02400000e+03, 4.09600000e+03, 1.63840000e+04,
       6.55360000e+04, 2.62144000e+05, 1.04857600e+06, 4.19430400e+06,
       1.67772160e+07, 6.71088640e+07, 2.68435456e+08, 1.07374182e+09])

## DepthFirst Search Algorithm for SAW Enumeration

**0**. Decide on a fixed local search order around a site. Initialize all variables.

**1**. Occupy the starting point, set $n = 1$, and set search direction index at this site as
$dir(1) = 1$.

**2**. Test if all directions around site n have already been searched.
  - If yes:
    - *Unoccupy* site n.
    - Decrement $n \Rightarrow n − 1$.   
    - If $n = 0$, you are done. Otherwise, repeat **2**.
  - If not, test the target in $dir(n)$ from site $n$ is *available*:
    - If it is:
        - Extend SAW to this site and add this walk to statistics.
        - Test if $n+1=n_{max}$.
            - If it has, increment $dir(n)$ and repeat 2.
            - If not:
                - Increment $dir(n)$.
                - Increment $n \Rightarrow n + 1$. 
                - *Occupy* the target site.
                - Set $dir(n) = 1$.
                - Repeat **2**.
        - If it is not, increment $dir(n)$ and repeat **2**.